In [14]:
import pandas as pd
import numpy as np

train = pd.read_csv("./pre_processed_train.csv",index_col="id")
test = pd.read_csv("./pre_processed_test.csv",index_col="id")

X=train.copy()
X_test=test.copy()


y=X.label
y_test=X_test.label

X=X.drop(['label'],axis=1)
X_test=X_test.drop(['label'],axis=1)

train.head()

,label,confidence,social_timestamp,social_karma,syntax_ari,lex_liwc_WC,lex_liwc_Analytic,lex_liwc_Clout,lex_liwc_Authentic,lex_liwc_Tone,...,Tokenized_Segment_1601,Tokenized_Segment_1602,Tokenized_Segment_1603,Tokenized_Segment_1604,Tokenized_Segment_1605,Tokenized_Segment_1606,Tokenized_Segment_1607,Tokenized_Segment_1608,Tokenized_Segment_1609,Tokenized_Segment_1610
id,,,,,,,,,,,,,,,,,,,,,
33181,1,0.8,1521614353,5,1.806818,116,72.64,15.04,89.26,1.00,...,0,0,0,0,0,0,0,0,0,0
2606,0,1.0,1527009817,4,9.429737,109,79.08,76.85,56.75,98.18,...,0,0,0,0,0,0,0,0,0,0
38816,1,0.8,1535935605,2,7.769821,167,33.80,76.38,86.24,25.77,...,0,0,0,0,0,0,0,0,0,0
239,1,0.6,1516429555,0,2.667798,273,2.98,15.25,95.42,79.26,...,0,0,0,0,0,0,0,0,0,0
1421,1,0.8,1539809005,24,7.554238,89,32.22,28.71,84.01,1.00,...,0,0,0,0,0,0,0,0,0,0


In [15]:
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

import warnings
warnings.filterwarnings(action='ignore', category=UserWarning)


In [16]:
# evaluate a given model using cross-validation
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
def evaluate_model(model, X, y):
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=42)
    model.fit(X_train, y_train)
    
    y_hat = model.predict(X_valid)
    
    return f1_score(y_valid,y_hat,average="binary")


In [17]:
### Optimizing parameters for XGBoost, GBM, LGBM which gave them highest base accuracy of 78%

##XGBoost

# optuna to optimize
import optuna
from sklearn.model_selection import cross_val_score

def xgb_objective(trial):
    
    n_estimators = trial.suggest_int("n_estimators", 300, 5000)
    max_depth = trial.suggest_int("max_depth", 1, 10)
    learning_rate = trial.suggest_float("learning_rate", 1e-6, 1e-1, log=True)
    min_child_weight = trial.suggest_int("min_child_weight", 1, 7, step=2)
    subsample = trial.suggest_float("subsample", 0.5, 1.0, step=0.1)
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.5, 1.0, step=0.1)
    
    model = XGBClassifier(n_estimators=n_estimators,
                         max_depth=max_depth,
                         learning_rate=learning_rate,
                         min_child_weight=min_child_weight,
                         colsample_bytree=colsample_bytree,
                         subsample=subsample,
                         n_jobs=-1, 
                         tree_method='gpu_hist', 
                         gpu_id=0)  
    return evaluate_model(model, X, y)

study = optuna.create_study(direction='maximize')
study.optimize(xgb_objective, n_trials=50)

[I 2021-08-29 14:55:53,818] A new study created in memory with name: no-name-8099f10d-57b5-43fb-a317-7fd88b2155ce


[14:55:54] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


KeyboardInterrupt: 

In [ ]:
print(study.best_trial)
print(study.best_trial.params)

In [ ]:
def gbm_objective(trial):
    
    n_estimators = trial.suggest_int("n_estimators", 300, 5000)
    max_depth = trial.suggest_int("max_depth", 1, 10)
    learning_rate = trial.suggest_float("learning_rate", 1e-6, 1e-1, log=True)
    subsample = trial.suggest_float("subsample", 0.5, 1.0, step=0.1)
    min_samples_split=trial.suggest_int("min_samples_split", 100, 1000,step=10)
    model = GradientBoostingClassifier(n_estimators=n_estimators,
                         max_depth=max_depth,
                         learning_rate=learning_rate,
                         subsample=subsample, 
                        min_samples_split=min_samples_split,
                         )  
    return evaluate_model(model, X, y)

study_gbm = optuna.create_study(study_name='gbm_optimze',direction='maximize')
study_gbm.optimize(gbm_objective, n_trials=50)

[I 2021-08-29 14:59:34,674] A new study created in memory with name: gbm_optimze
[I 2021-08-29 15:00:06,242] Trial 0 finished with value: 0.6987399770904925 and parameters: {'n_estimators': 699, 'max_depth': 9, 'learning_rate': 3.682545210834743e-05, 'subsample': 0.7, 'min_samples_split': 750}. Best is trial 0 with value: 0.6987399770904925.
[I 2021-08-29 15:01:58,923] Trial 1 finished with value: 0.7625201938610664 and parameters: {'n_estimators': 822, 'max_depth': 8, 'learning_rate': 0.0027409760166850036, 'subsample': 0.8, 'min_samples_split': 100}. Best is trial 1 with value: 0.7625201938610664.
[I 2021-08-29 15:04:11,669] Trial 2 finished with value: 0.7445255474452555 and parameters: {'n_estimators': 3777, 'max_depth': 7, 'learning_rate': 4.493602276424878e-05, 'subsample': 0.5, 'min_samples_split': 510}. Best is trial 1 with value: 0.7625201938610664.
[I 2021-08-29 15:04:43,346] Trial 3 finished with value: 0.6987399770904925 and parameters: {'n_estimators': 480, 'max_depth': 4,

In [ ]:
print(study_gbm.best_trial)
print(study_gbm.best_trial.params)

In [ ]:
def lgbm_objective(trial):
    
    num_leaves = trial.suggest_int("num_leaves", 24, 80)
    learning_rate = trial.suggest_float("learning_rate", 1e-6, 1e-1, log=True)
    feature_fraction=trial.suggest_float("feature_fraction", 0.1, 0,9)
    bagging_fraction=trial.suggest_float("bagging_fraction", 0.8, 1)
    max_depth=trial.suggest_int("max_depth",5,30)
    max_bin=trial.suggest_int("max_bin",20,90)
    max_data_in_leaf=trial.suggest_int("max_dat_in_leaf",20,80)
    min_sum_hessian_in_leaf=trial.suggest_int("min_sum_hessian_in_leaf",0,100)
    subsample = trial.suggest_float("subsample", 0.5, 1.0, step=0.1)
    model = LGBMClassifier(
                        application='binary',
                        eval_metric='auc',
                        num_leaves=num_leaves,
                        learning_rate=learning_rate,
                        bagging_fraction=bagging_fraction,
                        max_depth=max_depth,
                        max_bin=max_bin,
                        max_data_in_leaf=max_data_in_leaf,
                        min_sum_hessian_in_leaf=min_sum_hessian_in_leaf,
                        subsample=subsample,
                        early_stopping_rounds = 250,
                         tree_method='gpu_hist'
                        n_jobs=-1, 
                        gpu_id=0)  
    return evaluate_model(model, X, y)

study_lgbm = optuna.create_study(study_name='lgbm_optimze',direction='maximize')
study_lgbm.optimize(gbm_objective, n_trials=50)

In [ ]:
# best parameters for the models from Optuna

xgboost_parameters={'n_estimators': 843, 'max_depth': 7, 'learning_rate': 0.0034546899925579112, 'min_child_weight': 5, 'subsample': 0.8, 'colsample_bytree': 0.5}